In [142]:
import lxml.etree as ET
import hashlib

In [156]:
class TeiReader():
    def __init__(self, file):
        self.ns_tei = {'tei': "http://www.tei-c.org/ns/1.0"}
        self.ns_xml = {'xml': "http://www.w3.org/XML/1998/namespace"}
        self.file = file
        try:
            self.tree = ET.parse(file)
        except:
            None
        try:
            self.parsed_file = ET.tostring(self.tree, encoding="utf-8")
        except:
            self.parsed_file = "parsing didn't work"
    
    def find_elements(self, tei_element='placeName'):
        result = {'tei_element': tei_element}
        result['hits'] = self.tree.xpath('//tei:text//tei:{}'.format(tei_element), namespaces=self.ns_tei)
        result['nr_of_hits'] = len(result['hits'])
        return result
    
    def add_ids(self, tei_element='placeName', id_prefix='some', export=True, export_file="some"):
        hits = self.find_elements(tei_element)['hits']
        ids = []
        for x in hits:
            if x.text is None:
                break
            else:
                ref = hashlib.md5(x.text.encode('utf-8')).hexdigest()
            x.attrib['ref'] = "#{}_{}".format(id_prefix, ref)
            ids.append({x.text: x.attrib['ref']})
            if export:
                file = "{}.xml".format(export_file)
                with open(file, 'wb') as f:
                    f.write(ET.tostring(self.tree, pretty_print=True))
        return ids
        

In [157]:
hansi = TeiReader("teihencer-data/placeNoRef.xml")

In [162]:
persons = hansi.add_ids(tei_element='placeName', id_prefix='place', export_file='teihencer-data/new')